# Cкрейпинг информации о курсах ФКН ПМИ с портала hse.ru

Наша задача – собрать данные об оценивании на курсах ОП "Прикладная математика и информатика". Нужную для нас информацию можно найти на странице "Учебные курсы" соответствующей образовательной программы (https://www.hse.ru/ba/ami/courses/).

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Список курсов располагается на нескольких страницах, так что пройдемся по всем им и поскрейпим название курса и ссылку на страницу с описанием.

In [2]:
MAX_NUM_PAGES = 10
year = 2020
OK_RESPONSE_STATUS = 200
course_names = []
course_links = []

for page_num in range(MAX_NUM_PAGES):
    url = f'https://www.hse.ru/ba/ami/courses/page{page_num}.html?year={year}'
    try:
        response = requests.get(url, timeout=20)
    except:
        break
    if response.status_code != OK_RESPONSE_STATUS:
        break
        
    soup = BeautifulSoup(response.content, 'html.parser')
    courses = soup.find_all('a', 'link link_dark')
    
    if not courses:
        break
        
    page_course_names = [course.text.strip() for course in courses] 
    page_course_links = [course['href'] for course in courses]
    
    assert(len(page_course_names) == len(page_course_links))
    
    course_names += page_course_names
    course_links += page_course_links

Создадим DataFrame из двух столбцов: название предмета и ссылка на его страничку.

In [3]:
df = pd.DataFrame(
    {
        'name': course_names,
        'link': course_links
    }
)


In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None  

In [5]:
df.head(20)

,name,link
0,DevOps,https://www.hse.ru/ba/ami/courses/375265864.html
1,Автоматическая обработка текста,https://www.hse.ru/ba/ami/courses/339553234.html
2,Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html
3,Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html
4,Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html
5,Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html
6,Алгоритмы и структуры данных 2,https://www.hse.ru/ba/ami/courses/375294760.html
7,Алгоритмы и структуры данных 2 (углубленный курс),https://www.hse.ru/ba/ami/courses/375294728.html
8,Алгоритмы и структуры данных (углубленный курс),https://www.hse.ru/ba/ami/courses/394796012.html
9,Анализ данных в бизнесе,https://www.hse.ru/ba/ami/courses/375283606.html


In [6]:
df.shape

(136, 2)

Видим, что некоторые курсы дублируются, при чем содержание страниц одинаково (по крайней мере нужное нам), так что удалим повторения.

In [7]:
df = df.drop_duplicates(subset='name')
df = df.set_index('name')

In [8]:
df.head(20)

,link
name,
DevOps,https://www.hse.ru/ba/ami/courses/375265864.html
Автоматическая обработка текста,https://www.hse.ru/ba/ami/courses/339553234.html
Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html
Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html
Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html
Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html
Алгоритмы и структуры данных 2,https://www.hse.ru/ba/ami/courses/375294760.html
Алгоритмы и структуры данных 2 (углубленный курс),https://www.hse.ru/ba/ami/courses/375294728.html
Алгоритмы и структуры данных (углубленный курс),https://www.hse.ru/ba/ami/courses/394796012.html


In [9]:
df.shape

(105, 1)

Для каждого курса на его страничке найдем информацию о формуле оценки и запишем ее в новый столбец. Также учтем, что в ПУДе некоторых курсов, к сожалению, отсутствуют формулы оценки. Выкинем такие курсы из нашего датафрейма.

In [10]:
row_soups = []
for row in df.iloc:
    try:
        row_response = requests.get(row.link, timeout=20)
    except:
        break
    if response.status_code != OK_RESPONSE_STATUS:
        break
        
    row_soups.append(BeautifulSoup(row_response.content, 'html.parser'))   

In [11]:
formulas = []
for row_soup in row_soups:
    pud_sections = row_soup.find_all('div', 'pud__section')
    assessment_section = None
    for section in pud_sections:
        img = section.find('img', alt=True)
        if img and img['alt'] in ['Промежуточная аттестация', 'Interim Assessment']:
            assessment_section = section
            break
    if not assessment_section:
        formulas.append(None)
        continue
    formula = assessment_section.find_all('div', 'grey')[-1].text.strip()
    formulas.append(formula)

In [12]:
df['raw_formula'] = formulas
df = df.dropna()

In [13]:
df.head(10)

,link,raw_formula
name,,
DevOps,https://www.hse.ru/ba/ami/courses/375265864.html,0.33 * HW1 + 0.33 * HW2 + 0.34 * HW3
Автоматическая обработка текста,https://www.hse.ru/ba/ami/courses/339553234.html,"Пром1 = Округление(0.3 ДЗ1 + 0.3 ДЗ2 + 0.4 Проект1) <br />Пром2 = Округление(0.3 ДЗ3 + 0.3 ДЗ4 + 0.4 Проект2). <br />O<sub>вопросы</sub>=0,5 или 0 <br />Автомат: при 1/2 Округление (Пром1+Пром2) >= 8 автоматически выставляется оценка за Экзамен = 1/2 Округление (Пром1+Пром2).<br />\nИтоговая оценка по данной учебной дисциплине (округление арифметическое):<br />\nO<sub>итоговая</sub> = Минимум(0,3·Пром<sub>1</sub> + 0,3·Пром<sub>2</sub>+0,4·O<sub>экзамен</sub>+O<sub>ответы на вопросы</sub>, 10)"
Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html,0.4 * Literature Review/Introduction + 0.2 * аудиторная работа + 0.4 * самостоятельная работа
Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html,0.3 * Еженедельные домашние задания + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html,0.15 * Домашнее задание 1 + 0.15 * Домашнее задание 2 + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html,0.4 * Домашнее задание (4 модуль) + 0.33 * Промежуточная аттестация (2 модуль) + 0.07 * Работа на семинаре 1 + 0.2 * Экзамен
Алгоритмы и структуры данных 2,https://www.hse.ru/ba/ami/courses/375294760.html,0.15 * Домашнее задание + 0.15 * Домашнее задание + 0.3 * Работа на семинаре + 0.4 * Экзамен
Алгоритмы и структуры данных 2 (углубленный курс),https://www.hse.ru/ba/ami/courses/375294728.html,0.6 * Накопленная оценка + 0.4 * Экзамен
Алгоритмы и структуры данных (углубленный курс),https://www.hse.ru/ba/ami/courses/394796012.html,0.3 * Контесты + 0.15 * Контрольные + 0.25 * Листки + 0.3 * Экзамен


In [14]:
df.shape

(98, 2)

Некоторые описания оценивания содержат ошибки форматирования, в частности теги. Почистим это, используя регулярные выражения.

In [15]:
import re

In [16]:
def clean(s : str) -> str:
    html_tags_clean_pattern = re.compile('<[^а-я]*?>|&(\w+|#\d{1,6}|#x[\da-f]{1,6});')
    res = re.sub(html_tags_clean_pattern, '', s)
    return res.replace('\n', ' ').replace('\t', ' ')

In [17]:
df['formula'] = df['raw_formula'].apply(lambda s : clean(s))
df = df.drop(columns='raw_formula')

In [18]:
df

,link,formula
name,,
DevOps,https://www.hse.ru/ba/ami/courses/375265864.html,0.33 * HW1 + 0.33 * HW2 + 0.34 * HW3
Автоматическая обработка текста,https://www.hse.ru/ba/ami/courses/339553234.html,"Пром1 = Округление(0.3 ДЗ1 + 0.3 ДЗ2 + 0.4 Проект1) Пром2 = Округление(0.3 ДЗ3 + 0.3 ДЗ4 + 0.4 Проект2). Oвопросы=0,5 или 0 Автомат: при 1/2 Округление (Пром1+Пром2) >= 8 автоматически выставляется оценка за Экзамен = 1/2 Округление (Пром1+Пром2). Итоговая оценка по данной учебной дисциплине (округление арифметическое): Oитоговая = Минимум(0,3·Пром1 + 0,3·Пром2+0,4·Oэкзамен+Oответы на вопросы, 10)"
Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html,0.4 * Literature Review/Introduction + 0.2 * аудиторная работа + 0.4 * самостоятельная работа
Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html,0.3 * Еженедельные домашние задания + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html,0.15 * Домашнее задание 1 + 0.15 * Домашнее задание 2 + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html,0.4 * Домашнее задание (4 модуль) + 0.33 * Промежуточная аттестация (2 модуль) + 0.07 * Работа на семинаре 1 + 0.2 * Экзамен
Алгоритмы и структуры данных 2,https://www.hse.ru/ba/ami/courses/375294760.html,0.15 * Домашнее задание + 0.15 * Домашнее задание + 0.3 * Работа на семинаре + 0.4 * Экзамен
Алгоритмы и структуры данных 2 (углубленный курс),https://www.hse.ru/ba/ami/courses/375294728.html,0.6 * Накопленная оценка + 0.4 * Экзамен
Алгоритмы и структуры данных (углубленный курс),https://www.hse.ru/ba/ami/courses/394796012.html,0.3 * Контесты + 0.15 * Контрольные + 0.25 * Листки + 0.3 * Экзамен


Хотим теперь отпарсить формулы так, чтобы можно было вычислять значения оценки по списку значений ее компонент. Для начала заметим, что подавляющее большинство формул записано в довольно хорошем формате, то есть как

$$weight_1 * O_1 +  weight_2 * O_2 + \dots$$

Поработаем с ними, сложные формулы с длинными описаниями различных условий пока откинем. Сделаем это достаточно наивно: оставим только курсы с длиной формулы не более 200 символов.

In [19]:
from copy import deepcopy

In [20]:
df_short = deepcopy(df[df.formula.map(len) <= 200])
df_short.shape

(74, 2)

In [21]:
df_short.head(10)

,link,formula
name,,
DevOps,https://www.hse.ru/ba/ami/courses/375265864.html,0.33 * HW1 + 0.33 * HW2 + 0.34 * HW3
Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html,0.4 * Literature Review/Introduction + 0.2 * аудиторная работа + 0.4 * самостоятельная работа
Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html,0.3 * Еженедельные домашние задания + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html,0.15 * Домашнее задание 1 + 0.15 * Домашнее задание 2 + 0.2 * Контрольная работа + 0.5 * Экзамен
Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html,0.4 * Домашнее задание (4 модуль) + 0.33 * Промежуточная аттестация (2 модуль) + 0.07 * Работа на семинаре 1 + 0.2 * Экзамен
Алгоритмы и структуры данных 2,https://www.hse.ru/ba/ami/courses/375294760.html,0.15 * Домашнее задание + 0.15 * Домашнее задание + 0.3 * Работа на семинаре + 0.4 * Экзамен
Алгоритмы и структуры данных 2 (углубленный курс),https://www.hse.ru/ba/ami/courses/375294728.html,0.6 * Накопленная оценка + 0.4 * Экзамен
Алгоритмы и структуры данных (углубленный курс),https://www.hse.ru/ba/ami/courses/394796012.html,0.3 * Контесты + 0.15 * Контрольные + 0.25 * Листки + 0.3 * Экзамен
Анализ данных в бизнесе,https://www.hse.ru/ba/ami/courses/375283606.html,0.1 * Контрольная работа №1 + 0.1 * Контрольная работа №2 + 0.1 * Контрольная работа №3 + 0.5 * Практический проект (Командный проект) + 0.2 * Экзамен


Для хранения формулы напишем класс, который будем содержать в себе два поля: список весов и список названий элементов контроля.

In [22]:
from __future__ import annotations
import numpy as np
from typing import Tuple, Type


class Grade:

    def __init__(self, weights: Tuple[float], names: Tuple[str]) -> None:
        assert(len(weights) == len(names))
        self.weights = np.array(weights)
        self.names = np.array(names)

    def __call__(self, values: Tuple[float]) -> float:
        assert(len(self.weights) == len(values))
        return sum(self.weights * np.array(values))

    def __repr__(self) -> str:
        return 'Grade(' + 'weights: ' + str(self.weights) + ', names: ' + str(self.names) + ')'

    @staticmethod
    def from_string(formula: str) -> Grade:
        # splitting into terms by '+'
        terms = tuple(
            map(str.strip,
                formula.split('+')
                )
        )
        # splitting each terms into weight and mark by '*'
        weights_names = tuple(
            map(lambda s: s.split('*'),
                terms)
        )
        # tranforming data into two tuples : weights of marks and names of marks
        weights, names = tuple(
            zip(*weights_names)
        )
        # unifying numbers format
        weights = tuple(
            map(
                float,
                map(
                    lambda s: s.replace(',', '.'),
                    map(str.strip,
                        weights
                        )
                )
            )
        )
        # stripping marks' names
        names = tuple(
            map(
                str.strip,
                names
            )
        )
        return Grade(weights, names)


(Манипуляции с записью класса в файл и его импортированием связаны с тем, что в дальнейшем мы будем использовать этот класс в веб-приложении и чтобы данные сериализовались, а потом подгружались корректно необходимо, чтобы класс сразу лежал в файле grade.py)

Создадим новый столбец с объектом класса Grade.

In [23]:
grades = []
for row in df_short.iloc:
    grades.append(Grade.from_string(row.formula))

In [24]:
df_short['grade'] = grades

In [25]:
df_short.tail(10)

,link,formula,grade
name,,,
Semantic Technologies,https://www.hse.ru/ba/ami/courses/339555772.html,0.4 * Exam + 0.6 * In-class tasks/ homeworks,"Grade(weights: [0.4 0.6], names: ['Exam' 'In-class tasks/ homeworks'])"
Systems Analysis,https://www.hse.ru/ba/ami/courses/375292720.html,0.5 * Oexam + 0.1 * Oproject1 + 0.15 * Oproject2 + 0.25 * Otest,"Grade(weights: [0.5 0.1 0.15 0.25], names: ['Oexam' 'Oproject1' 'Oproject2' 'Otest'])"
How to Win a Data Science Competition: Learn from Top Kagglers,https://www.hse.ru/ba/ami/courses/375286436.html,0.7 * Competition + 0.3 * Online course,"Grade(weights: [0.7 0.3], names: ['Competition' 'Online course'])"
Choice and Decision Theory,https://www.hse.ru/ba/ami/courses/339555682.html,0.6 * final exam + 0.2 * homework + 0.2 * mid-term exam,"Grade(weights: [0.6 0.2 0.2], names: ['final exam' 'homework' 'mid-term exam'])"
Theory of Computation,https://www.hse.ru/ba/ami/courses/339557398.html,0.35 * Colloquium + 0.3 * Exam + 0.35 * Homework,"Grade(weights: [0.35 0.3 0.35], names: ['Colloquium' 'Exam' 'Homework'])"
Теория и практика многопоточной синхронизации,https://www.hse.ru/ba/ami/courses/375301715.html,0.5 * Домашние задания + 0.5 * Домашние задания,"Grade(weights: [0.5 0.5], names: ['Домашние задания' 'Домашние задания'])"
Теория отказоустойчивых распределенных систем,https://www.hse.ru/ba/ami/courses/339559772.html,0.7 * Домашние задания + 0.3 * Экзамен,"Grade(weights: [0.7 0.3], names: ['Домашние задания' 'Экзамен'])"
Statistical Learning Theory,https://www.hse.ru/ba/ami/courses/375266752.html,0.3 * Colloqium + 0.35 * Exam + 0.35 * Homework,"Grade(weights: [0.3 0.35 0.35], names: ['Colloqium' 'Exam' 'Homework'])"
Философия науки,https://www.hse.ru/ba/ami/courses/375266992.html,"0,6 * (оценка за экзамен) + 0,4 * (сумма баллов за текущую работу, но не более 10)","Grade(weights: [0.6 0.4], names: ['(оценка за экзамен)' '(сумма баллов за текущую работу, но не более 10)'])"


Сохраним получившийся датафрейм используя модуль pickle для сериализации объектов класса Grade.

In [26]:
df_short.to_pickle('assessment_db.pkl')

Проверим, что все работает и данные выгружаются в нужном формате.

In [27]:
db = pd.read_pickle('assessment_db.pkl')
db.head()

,link,formula,grade
name,,,
DevOps,https://www.hse.ru/ba/ami/courses/375265864.html,0.33 * HW1 + 0.33 * HW2 + 0.34 * HW3,"Grade(weights: [0.33 0.33 0.34], names: ['HW1' 'HW2' 'HW3'])"
Академическое письмо на английском языке,https://www.hse.ru/ba/ami/courses/339581328.html,0.4 * Literature Review/Introduction + 0.2 * аудиторная работа + 0.4 * самостоятельная работа,"Grade(weights: [0.4 0.2 0.4], names: ['Literature Review/Introduction' 'аудиторная работа'\n 'самостоятельная работа'])"
Алгебра,https://www.hse.ru/ba/ami/courses/394774683.html,0.3 * Еженедельные домашние задания + 0.2 * Контрольная работа + 0.5 * Экзамен,"Grade(weights: [0.3 0.2 0.5], names: ['Еженедельные домашние задания' 'Контрольная работа' 'Экзамен'])"
Алгебра (углубленный курс),https://www.hse.ru/ba/ami/courses/394799196.html,0.15 * Домашнее задание 1 + 0.15 * Домашнее задание 2 + 0.2 * Контрольная работа + 0.5 * Экзамен,"Grade(weights: [0.15 0.15 0.2 0.5 ], names: ['Домашнее задание 1' 'Домашнее задание 2' 'Контрольная работа' 'Экзамен'])"
Алгоритмы и структуры данных,https://www.hse.ru/ba/ami/courses/394764539.html,0.4 * Домашнее задание (4 модуль) + 0.33 * Промежуточная аттестация (2 модуль) + 0.07 * Работа на семинаре 1 + 0.2 * Экзамен,"Grade(weights: [0.4 0.33 0.07 0.2 ], names: ['Домашнее задание (4 модуль)' 'Промежуточная аттестация (2 модуль)'\n 'Работа на семинаре 1' 'Экзамен'])"


In [28]:
type(db.iloc[0]['grade'])

__main__.Grade